<a href="https://colab.research.google.com/github/ashivashankars/Team_4Musketeers_capstone_project/blob/Archana_Jobs_with_feature_extraction/Jobs_with_Feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install -q openai
!pip install -q PyPDF2
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 30.2 MB/s eta 0:00:00


In [13]:
import os
import json
import time
import sys
import re
import pandas as pd
import requests
import faiss
import numpy as np
import gradio

from openai import OpenAI
import PyPDF2
from google.colab import userdata
from google.colab import files
from google.colab import drive
from bs4 import BeautifulSoup

In [14]:
# Define Google Drive path for caching
DRIVE_PATH = '/content/drive/MyDrive/job_matching_cache'

# --- 1. SECURE API KEY HANDLING ---
try:
    # Tries to get key from Colab Secrets (Left sidebar > Key icon)
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if not OPENAI_API_KEY:
        raise ValueError("OPENAI_API_KEY not found in Colab Secrets.")
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("OpenAI API Key loaded securely from Colab Secrets.")
except Exception as e:
    print(f"Error: {e}")
    print("Please click the Key icon on the left, add 'OPENAI_API_KEY', and enable notebook access.")
    sys.exit(1)

# --- 2. CONFIGURATION & HELPER FUNCTIONS ---
# (Manual file upload removed as it is handled by the Gradio App)

FIELD_CONFIG = {
    "graduation_date":      {"id": 1, "text": "Expected Graduation Date", "mode": "interactive"},
    "current_degree_major": {"id": 2, "text": "Current Degree AND Major (e.g. MS in CS)", "mode": "interactive"},
    "current_degree_gpa":   {"id": 3, "text": "GPA (For Current Degree Only)", "mode": "interactive"},
    "us_citizenship":       {"id": 4, "text": "Are you a US Citizen? (If No, specify Visa Type & Sponsorship)", "mode": "interactive"},
    "programming_languages": {"id": 5, "text": "Programming Languages", "mode": "extract"},
    "experience_software":   {"id": 6, "text": "Work/Project Experience Summary", "mode": "extract"},
    "tools_frameworks":      {"id": 7, "text": "Tools & Frameworks", "mode": "extract"},
    "leadership":            {"id": 8, "text": "Leadership Experience", "mode": "extract"},
    "job_preference":        {"id": 9, "text": "Looking for Full-time / Internship / Both?", "mode": "interactive"},
    "impact_outcomes":       {"id": 10,"text": "Quantifiable Impact & Key Achievements", "mode": "extract"}
}

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page_num in range(len(reader.pages)):
                text += reader.pages[page_num].extract_text()
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return None
    return text

def save_embeddings_and_index(index, prepared_job_data_to_save, drive_path):
    print("\n--- Saving embeddings and FAISS index to Google Drive ---")
    drive.mount('/content/drive', force_remount=True)

    os.makedirs(drive_path, exist_ok=True)

    faiss_index_path = os.path.join(drive_path, 'faiss_index.bin')
    job_data_path = os.path.join(drive_path, 'prepared_job_data.json')

    try:
        faiss.write_index(index, faiss_index_path)
        print(f"FAISS index saved to {faiss_index_path}")

        # Prepare prepared_job_data for JSON serialization
        # Convert numpy arrays (job_embedding) to lists
        serializable_job_data = []
        for job_dict in prepared_job_data_to_save:
            temp_job_dict = job_dict.copy()
            if 'job_embedding' in temp_job_dict and isinstance(temp_job_dict['job_embedding'], np.ndarray):
                temp_job_dict['job_embedding'] = temp_job_dict['job_embedding'].tolist()
            serializable_job_data.append(temp_job_dict)

        with open(job_data_path, 'w') as f:
            json.dump(serializable_job_data, f, indent=4)
        print(f"Prepared job data saved to {job_data_path}")
    except Exception as e:
        print(f"Error saving cache files: {e}")

def load_embeddings_and_index(drive_path):
    print("\n--- Attempting to load embeddings and FAISS index from Google Drive ---")
    drive.mount('/content/drive', force_remount=True)

    faiss_index_path = os.path.join(drive_path, 'faiss_index.bin')
    job_data_path = os.path.join(drive_path, 'prepared_job_data.json')

    loaded_index = None
    loaded_prepared_job_data = None

    try:
        if os.path.exists(faiss_index_path) and os.path.exists(job_data_path):
            loaded_index = faiss.read_index(faiss_index_path)
            print(f"FAISS index loaded from {faiss_index_path}")

            with open(job_data_path, 'r') as f:
                loaded_prepared_job_data = json.load(f)
            print(f"Prepared job data loaded from {job_data_path}")

            # Convert job_embedding back to numpy array
            for job_dict in loaded_prepared_job_data:
                if 'job_embedding' in job_dict and isinstance(job_dict['job_embedding'], list):
                    job_dict['job_embedding'] = np.array(job_dict['job_embedding'], dtype='float32')

            print("Cache loaded successfully.")
            return loaded_index, loaded_prepared_job_data
        else:
            raise FileNotFoundError("Cache files not found.")
    except FileNotFoundError:
        print("Cache files (FAISS index or prepared job data) not found in Google Drive. Will generate new ones.")
        return None, None
    except Exception as e:
        print(f"Error loading cache files: {e}")
        return None, None

OpenAI API Key loaded securely from Colab Secrets.


In [15]:
print("------------------------------")
print("--- Pre-loading Job Data and Initializing FAISS Index (with Caching) ---")

# Attempt to load from cache first
try:
    loaded_index, loaded_prepared_job_data = load_embeddings_and_index(DRIVE_PATH)
except Exception as e:
    print(f"Error loading from Google Drive cache: {e}. Proceeding without cache.")
    loaded_index, loaded_prepared_job_data = None, None

if loaded_index is not None and loaded_prepared_job_data is not None:
    index = loaded_index
    prepared_job_data = loaded_prepared_job_data
    # Ensure job_type is added if loading from an older cache that didn't save it
    for j in prepared_job_data:
        if 'job_type' not in j:
            r = j.get('role', '')
            if re.search(r'intern', r, re.IGNORECASE): j['job_type'] = 'Intern'
            elif re.search(r'new grad|graduate', r, re.IGNORECASE): j['job_type'] = 'New Grad'
            else: j['job_type'] = 'Full-time'
    print("Data and FAISS index successfully loaded from Google Drive cache.")
else:
    print("Cache not found or failed to load. Proceeding with fresh data generation.")
    # --- 1. Fetch Job Postings and Parse ---
    # Define the GitHub raw URL
    github_raw_url = "https://raw.githubusercontent.com/SimplifyJobs/Summer2026-Internships/dev/README.md"

    # Fetch the content of the README.md file
    try:
        response = requests.get(github_raw_url)
        response.raise_for_status() # Raise an exception for HTTP errors
        readme_content = response.text
        print(f"Successfully fetched README content from {github_raw_url}")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching README content: {e}")
        readme_content = "" # Initialize with empty string on error

    # Define a regular expression pattern to find level 2 headings
    heading_pattern = re.compile(r"^##\s(.+)$", re.MULTILINE)

    # Find all matching headings in the readme_content
    section_headings = heading_pattern.findall(readme_content)

    # Initialize an empty dictionary to store extracted DataFrames
    all_sections_data = {}

    # Iterate through each section heading to extract its content
    for i, current_heading in enumerate(section_headings):
        current_heading_full = f"## {current_heading}"
        start_index = readme_content.find(current_heading_full)

        if start_index == -1:
            continue

        end_index = -1
        if i + 1 < len(section_headings):
            next_heading_full = f"## {section_headings[i+1]}"
            end_index = readme_content.find(next_heading_full, start_index + len(current_heading_full))

        if end_index != -1:
            section_content = readme_content[start_index + len(current_heading_full):end_index].strip()
        else:
            section_content = readme_content[start_index + len(current_heading_full):].strip()

        section_dfs = [] # To store multiple tables if a section has them

        soup = BeautifulSoup(section_content, 'lxml')
        tables = soup.find_all('table')

        for table in tables:
            headers = []
            if table.find('thead'):
                for th in table.find('thead').find_all('th'):
                    headers.append(th.get_text(strip=True))

            data_rows = []
            if table.find('tbody'):
                for tr in table.find('tbody').find_all('tr'):
                    row_values = []
                    for idx, td in enumerate(tr.find_all('td')):
                        if headers and idx < len(headers) and headers[idx] == 'Application':
                            link = td.find('a')
                            if link and 'href' in link.attrs:
                                row_values.append(link['href'])
                            else:
                                row_values.append('')
                        else:
                            row_values.append(td.get_text(strip=True))

                    if len(headers) > 0:
                        if len(row_values) > len(headers):
                            row_values = row_values[:len(headers)]
                        elif len(row_values) < len(headers):
                            row_values.extend([''] * (len(headers) - len(row_values))) # Pad if too few
                        data_rows.append(row_values)

            if headers and data_rows:
                df = pd.DataFrame(data_rows, columns=headers)
                section_dfs.append(df)

        if section_dfs:
            all_sections_data[current_heading] = section_dfs[0]
        else:
            all_sections_data[current_heading] = None
    print(f"Extracted data for {len(all_sections_data)} sections from GitHub README.")

    # --- 2. Prepare Job Data for Embedding ---
    prepared_job_data = []

    for section_name, df in all_sections_data.items():
        if df is not None:
            df_normalized = df.copy()
            df_normalized.columns = df_normalized.columns.str.lower()

            # FIXED: Renamed loop variable from 'index' to 'idx' to avoid shadowing the global FAISS index variable
            for idx, row in df_normalized.iterrows():
                company = row['company'] if 'company' in row and pd.notna(row['company']) and str(row['company']).strip() != '' else 'N/A'
                role = row['role'] if 'role' in row and pd.notna(row['role']) and str(row['role']).strip() != '' else 'N/A'
                location = row['location'] if 'location' in row and pd.notna(row['location']) and str(row['location']).strip() != '' else 'N/A'
                application_link = row['application'] if 'application' in row and pd.notna(row['application']) and str(row['application']).strip() != '' else 'N/A'

                embedding_text_parts = [
                    f"Company: {company}",
                    f"Role: {role}",
                    f"Location: {location}",
                    f"Section: {section_name}"
                ]
                embedding_text = ". ".join(embedding_text_parts)

                job_details = row.to_dict()
                job_details['application_link'] = application_link

                if 'application' in job_details:
                    del job_details['application']

                job_details['section'] = section_name
                job_details['description'] = 'N/A'
                job_details['embedding_text'] = embedding_text

                # Assign job_type based on role, to be saved with the job_details
                r = job_details.get('role', '')
                if re.search(r'intern', r, re.IGNORECASE): job_details['job_type'] = 'Intern'
                elif re.search(r'new grad|graduate', r, re.IGNORECASE): job_details['job_type'] = 'New Grad'
                else: job_details['job_type'] = 'Full-time'

                prepared_job_data.append(job_details)
    print(f"Prepared {len(prepared_job_data)} job entries for embedding.")

    # --- 3. Generate Embeddings for Job Postings ---
    EMBEDDING_MODEL = "text-embedding-ada-002" # Define the embedding model
    job_embeddings = []

    for i, job in enumerate(prepared_job_data):
        try:
            job_embedding_response = client.embeddings.create(
                model=EMBEDDING_MODEL,
                input=job['embedding_text']
            )
            job['job_embedding'] = job_embedding_response.data[0].embedding
            job_embeddings.append(job['job_embedding'])
            if i % 100 == 0: # Print progress every 100 jobs
                print(f"Generated embedding for job {i+1}/{len(prepared_job_data)}")
        except Exception as e:
            print(f"Error generating embedding for job {i+1}: {e}")
            job['job_embedding'] = None
    print(f"Generated embeddings for {len(job_embeddings)} job postings.")

    # --- 4. Initialize FAISS Index and Add Job Embeddings ---
    embedding_dimension = 1536 # OpenAI's 'text-embedding-ada-002' dimension is 1536
    index = faiss.IndexFlatL2(embedding_dimension)

    valid_job_embeddings = [job['job_embedding'] for job in prepared_job_data if 'job_embedding' in job and job['job_embedding'] is not None]
    job_embeddings_np = np.array(valid_job_embeddings).astype('float32')

    if len(job_embeddings_np) > 0:
        index.add(job_embeddings_np)
        print(f"FAISS index initialized and {index.ntotal} job embeddings added.")
    else:
        print("No valid job embeddings to add to the FAISS index.")

    # Save the newly generated data to cache
    save_embeddings_and_index(index, prepared_job_data, DRIVE_PATH)

print("Pre-loading complete.")
print("------------------------------")

# Verification steps:
print(f"Number of job entries in prepared_job_data: {len(prepared_job_data)}")
print(f"Total vectors in FAISS index: {index.ntotal}")

------------------------------
--- Pre-loading Job Data and Initializing FAISS Index (with Caching) ---

--- Attempting to load embeddings and FAISS index from Google Drive ---
Mounted at /content/drive
FAISS index loaded from /content/drive/MyDrive/job_matching_cache/faiss_index.bin
Prepared job data loaded from /content/drive/MyDrive/job_matching_cache/prepared_job_data.json
Cache loaded successfully.
Data and FAISS index successfully loaded from Google Drive cache.
Pre-loading complete.
------------------------------
Number of job entries in prepared_job_data: 1140
Total vectors in FAISS index: 1140


In [16]:
print("--- Launching Self-Contained Gradio App (Master Fix) ---")

# --- 1. Configuration & Setup ---

# Define Configuration for Extraction
FIELD_CONFIG = {
    "graduation_date":      {"id": 1, "text": "Expected Graduation Date", "mode": "interactive"},
    "current_degree_major": {"id": 2, "text": "Current Degree AND Major (e.g. MS in CS)", "mode": "interactive"},
    "current_degree_gpa":   {"id": 3, "text": "GPA (For Current Degree Only)", "mode": "interactive"},
    "us_citizenship":       {"id": 4, "text": "Are you a US Citizen? (If No, specify Visa Type & Sponsorship)", "mode": "interactive"},
    "programming_languages": {"id": 5, "text": "Programming Languages", "mode": "extract"},
    "experience_software":   {"id": 6, "text": "Work/Project Experience Summary", "mode": "extract"},
    "tools_frameworks":      {"id": 7, "text": "Tools & Frameworks", "mode": "extract"},
    "leadership":            {"id": 8, "text": "Leadership Experience", "mode": "extract"},
    "job_preference":        {"id": 9, "text": "Looking for Full-time / Internship / Both?", "mode": "interactive"},
    "impact_outcomes":       {"id": 10,"text": "Quantifiable Impact & Key Achievements", "mode": "extract"}
}

# Setup OpenAI Client
try:
    if 'client' not in globals():
        OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
        if not OPENAI_API_KEY:
             print("WARNING: OPENAI_API_KEY not found in secrets. App may fail.")
        else:
             client = OpenAI(api_key=OPENAI_API_KEY)
except Exception as e:
    print(f"Client Init Error: {e}")

# Drive Path
DRIVE_PATH = '/content/drive/MyDrive/job_matching_cache'

# --- 2. Helper Functions (Extraction & Search) ---

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page_num in range(len(reader.pages)):
                text += reader.pages[page_num].extract_text()
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return None
    return text

def extract_profile_from_resume(resume_pdf_path, client):
    resume_text = extract_text_from_pdf(resume_pdf_path)
    if not resume_text:
        return {}, ""

    openai_prompt_content = f"""
    Analyze the resume text. Extract fields into JSON.
    STRICT RULES:
    - LEADERSHIP: List of objects (role, organization, description) or "NULL".
    - PREFERENCE: "Internship", "Full-time", or "Both".
    - NULL: Use "NULL" string if missing.

    Required JSON:
    {{
        "graduation_date": "value/NULL", "current_degree_major": "value/NULL", "current_degree_gpa": "value/NULL",
        "us_citizenship": "value/NULL", "programming_languages": "value/NULL", "experience_software": "value/NULL",
        "tools_frameworks": "value/NULL", "leadership": [...], "job_preference": "value/NULL", "impact_outcomes": "value/NULL"
    }}

    RESUME:
    {resume_text}
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            response_format={"type": "json_object"},
            messages=[{"role": "user", "content": openai_prompt_content}]
        )
        extracted_data = json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"LLM Extraction Error: {e}")
        extracted_data = {k: "NULL" for k in FIELD_CONFIG.keys()}

    final_output = {}
    for key in FIELD_CONFIG.keys():
        val = extracted_data.get(key, "NULL")
        final_output[key] = "Not Found in Resume" if val in ["NULL", None, ""] else val

    profile_parts = []
    for k, v in final_output.items():
        if v != "Not Found in Resume":
            if isinstance(v, list) and k == "leadership":
                 s = [f"{i.get('role','')}" for i in v]
                 profile_parts.append(f"{k}: {'; '.join(s)}")
            else:
                 profile_parts.append(f"{k}: {v}")
    return final_output, ". ".join(profile_parts)

def hybrid_search_jobs(resume_embedding, candidate_preferences, all_jobs):
    pref = str(candidate_preferences.get('job_preference', '')).lower()
    grad_date_str = str(candidate_preferences.get('graduation_date', '')).lower()

    allowed_types = set()
    if 'internship' == pref:
        allowed_types.add('Intern')
    elif 'full-time' == pref:
        allowed_types.update(['Full-time', 'New Grad'])
    elif 'both' == pref:
        allowed_types.update(['Intern', 'Full-time', 'New Grad'])
    else: # This 'else' covers cases where pref is not explicitly internship/full-time/both, or is extracted as 'NULL' etc.
        # Smart Defaults logic, now used only if preference is unclear/missing
        import datetime
        curr_year = datetime.datetime.now().year
        years = [int(y) for y in re.findall(r'\b\d{4}\b', grad_date_str)]
        if years and max(years) > curr_year:
             allowed_types = {'Intern', 'New Grad'}
        else:
             allowed_types = {'Full-time', 'New Grad'}

    valid_jobs = [j for j in all_jobs if j.get('job_embedding') is not None and j.get('job_type', 'Full-time') in allowed_types]
    if not valid_jobs: return []

    resume_vec = np.array(resume_embedding, dtype='float32').flatten()
    job_matrix = np.array([j['job_embedding'] for j in valid_jobs], dtype='float32')

    sims = np.dot(job_matrix, resume_vec) / (np.linalg.norm(resume_vec) * np.linalg.norm(job_matrix, axis=1) + 1e-10)

    results = [{'job_details': j, 'cosine_similarity': float(s)} for j, s in zip(valid_jobs, sims)]
    results.sort(key=lambda x: x['cosine_similarity'], reverse=True)
    return results

def find_matches_with_criteria(verified_prefs, profile_text, client, prepared_jobs):
    try:
        emb = client.embeddings.create(model="text-embedding-ada-002", input=profile_text).data[0].embedding
    except: return [], 0.0

    ranked = hybrid_search_jobs(emb, verified_prefs, prepared_jobs)
    if not ranked: return [], 0.0

    thresh = np.percentile([r['cosine_similarity'] for r in ranked], 95)
    return [r for r in ranked if r['cosine_similarity'] >= thresh], thresh

def augment_job_with_llm(job_match, profile_text, client):
    job = job_match['job_details']
    try:
        res = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role":"user", "content": f"Explain why candidate fits job in 2 sentences.\nCandidate: {profile_text}\nJob: {job.get('embedding_text','')}"}],
            max_tokens=100
        )
        job_match['llm_fit_summary'] = res.choices[0].message.content.strip()
    except:
        job_match['llm_fit_summary'] = "Analysis failed."
    return job_match

# --- 3. Gradio Logic ---

def extract_step(pdf_file):
    if not pdf_file: return "Both", "", "", {}, "", "", gradio.update(visible=False)

    try:
        # Load data if needed
        global prepared_job_data
        if 'prepared_job_data' not in globals() or prepared_job_data is None:
             # Minimal Load Logic reuse
             from google.colab import drive
             drive.mount('/content/drive', force_remount=True)
             with open(os.path.join(DRIVE_PATH, 'prepared_job_data.json'), 'r') as f:
                 prepared_job_data = json.load(f)
             for j in prepared_job_data:
                 if 'job_embedding' in j: j['job_embedding'] = np.array(j['job_embedding'], dtype='float32')
                 # The job_type should now be loaded from cache. This block ensures it's there if cache is old.
                 if 'job_type' not in j: # Auto-tag
                     r = j.get('role', '')
                     if re.search(r'intern', r, re.IGNORECASE): j['job_type'] = 'Intern'
                     elif re.search(r'new grad|graduate', r, re.IGNORECASE): j['job_type'] = 'New Grad'
                     else: j['job_type'] = 'Full-time'

        final_output, profile_text = extract_profile_from_resume(pdf_file.name, client)

        pref = "Both"
        raw_pref = final_output.get("job_preference", "").lower()
        if "intern" in raw_pref: pref = "Internship"
        elif "full" in raw_pref: pref = "Full-time"

        # Determine initial visibility of visa type textbox
        initial_visa_visible = "No (Specify Visa Type)" in final_output.get("us_citizenship", "")
        initial_visa_value = final_output.get("us_citizenship", "").replace("No (Specify Visa Type) - ", "") if initial_visa_visible else ""

        msg = "### ✅ Step 1 Complete.\nVerify details below, then click **'Find Matching Jobs'**."
        return (
            gradio.update(value=pref, interactive=True),
            gradio.update(value=final_output.get("current_degree_gpa", ""), interactive=True),
            gradio.update(value=final_output.get("us_citizenship", ""), interactive=True),
            final_output, profile_text, msg,
            gradio.update(value=initial_visa_value, visible=initial_visa_visible)
        )
    except Exception as e:
        return "Both", "Error", "Error", {}, "", f"### ⚠️ Error in Step 1:\n{str(e)}", gradio.update(visible=False)

def display_paginated(matches, page, thresh, text, progress=gradio.Progress()):
    start = page * 5
    batch = matches[start:start+5]
    if not batch: return ("No more jobs.", page, matches) if page > 0 else ("No matches.", 0, matches)

    out = ""
    if page == 0:
        out += f"### 🎯 Found {len(matches)} matches (Top 5% shown).\n---\n\n"

    for i, m in enumerate(batch):
        progress(None, desc=f"Analyzing {i+1}/5...")
        if 'llm_fit_summary' not in m: m = augment_job_with_llm(m, text, client)

        job = m['job_details']
        out += f"### {start+i+1}. {job.get('company','N/A')} - {job.get('role','N/A')}\n"
        out += f"**Loc:** {job.get('location','N/A')} | **Sim:** {m['cosine_similarity']*100:.1f}%\n"
        out += f"**Fit:** {m.get('llm_fit_summary','')}\n"
        out += f"[Apply]({job.get('application_link','#')})\n\n---\n\n"
    return out, page+1, matches

def search_step(pref, gpa, cit, visa_type, p_dict, p_text):
    if not p_dict: return "Upload resume first.", [], 0, 0.0, {}, ""

    # Update citizenship with visa type if applicable
    if cit == "No (Specify Visa Type)" and visa_type:
        full_citizenship_string = f"{cit} - {visa_type}"
    else:
        full_citizenship_string = cit

    p_dict.update({"job_preference": pref, "current_degree_gpa": gpa, "us_citizenship": full_citizenship_string})

    matches, thresh = find_matches_with_criteria(p_dict, p_text, client, prepared_job_data)
    if not matches: return "No matches found.", [], 0, 0.0, p_dict, p_text

    txt, pg, updated_matches = display_paginated(matches, 0, thresh, p_text)
    return txt, updated_matches, pg, thresh, p_dict, p_text

def toggle_visa_textbox(choice):
    if choice == "No (Specify Visa Type)":
        return gradio.update(visible=True, interactive=True)
    else:
        return gradio.update(visible=False, interactive=False)


# --- 4. Launch App ---
try: demo.close()
except: pass

with gradio.Blocks() as demo:
    gradio.Markdown("# 🎯 Resume Matcher (Master Fix)")

    state_dict = gradio.State({})
    state_text = gradio.State("")
    state_jobs = gradio.State([])
    state_page = gradio.State(0)
    state_thresh = gradio.State(0.0)

    with gradio.Row():
        with gradio.Column():
            f_in = gradio.File(label="1. Upload PDF")
            p_in = gradio.Radio(["Internship", "Full-time", "Both"], label="Preference")
            g_in = gradio.Textbox(label="GPA")
            c_in = gradio.Radio(["Yes", "No (Specify Visa Type)"], label="Citizenship")
            c_visa_in = gradio.Textbox(label="Visa Type", visible=False, interactive=False)
            btn = gradio.Button("2. Find Jobs", variant="primary")
        with gradio.Column():
            res_out = gradio.Markdown("Waiting for upload...")
            load_btn = gradio.Button("Load More")

    f_in.upload(extract_step, [f_in], [p_in, g_in, c_in, state_dict, state_text, res_out, c_visa_in])
    c_in.change(toggle_visa_textbox, [c_in], [c_visa_in])
    btn.click(search_step, [p_in, g_in, c_in, c_visa_in, state_dict, state_text], [res_out, state_jobs, state_page, state_thresh, state_dict, state_text])
    load_btn.click(display_paginated, [state_jobs, state_page, state_thresh, state_text], [res_out, state_page, state_jobs])

demo.launch(debug=False, share=True)
print("Master App Launched.")

--- Launching Self-Contained Gradio App (Master Fix) ---
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c81c696aa981acc346.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Master App Launched.
